In [1]:
import time

# Importing DASK libraries for using DASK for parallel mapping
from dask.distributed import Client, LocalCluster
from dask import visualize

# import the mapping module
from fldpln.mapping import *

In [2]:
# tiled library folder that may have more than one tiled libraries
libFolder =  'data\\tmp\\tile' # wildcat

# Set output folder
outputFolder = 'data\\maps'

In [5]:
# select the libraries under tiled library folder to generate maps
libs2Map = ['']

# set the special map wanted
# 'MinDtf','NumOfFsps','Depression', or a real number (for example 20.5, in foot for KS libraries) representing constant DOF at all the FSPs
specialDof = 'MinDtf' # Wildcat 3DEP DEM has a vertical unit of meters

# set up output map-folder under the outputFolder
outMapFolderName = 'mindtf_0905'

# Create folders for storing temp and output map files
outMapFolder,scratchFolder = CreateFolders(outputFolder,'scratch',outMapFolderName)

# whether mosaic tiles as a single COG
mosaicTiles = True #True #False

# Using LocalCluster by default
useLocalCluster = False
numOfWorkers = round(0.8*os.cpu_count())

In [11]:
# show mapping info
print(f'Tiled FLDPLN library folder: {libFolder}')
print(f'Map folder: {outMapFolder}')
# libs to map
print(f'Libraries to map: {libs2Map}')

# check running time
startTimeAllLibs = time.time()

# create a local cluster to speed up the mapping. Must be run inside "if __name__ == '__main__'"!!!
if useLocalCluster:
    # cluster = LocalCluster(n_workers=4,processes=False)
    try:
        print('Start a LocalCluster ...')
        # NOTE: set worker space (i.e., local_dir) to a folder that the LocalCluster can access. When run the script through a scheduled task, 
        # the system uses C:\Windows\system32 by default, which a typical user doesn't have the access!
        cluster = LocalCluster(n_workers=numOfWorkers,memory_limit='32GB',local_dir="D:/projects_new/fldpln/tools") # for KARS production server (192G RAM & 8 cores)
        # cluster = LocalCluster(n_workers=numOfWorkers,processes=False) # for KARS production server (192G RAM & 8 cores)
        # print('Watch workers at: ',cluster.dashboard_link)
        print(f'Number of workers: {numOfWorkers}')
        client = Client(cluster)
        # print scheduler info
        # print(client.scheduler_info())
    except:
        print('Cannot create a LocalCLuster!')
        useLocalCluster = False

# dict to store lib processing time
libTime={}

# map each library
for libName in libs2Map:
    # check running time
    startTime = time.time()
    
    # mapping flood depth
    if useLocalCluster:
        print(f'Map [{libName}] using LocalCLuster ...')
        # generate a DAG
        dag,dagRoot=MapFloodDepthWithTilesAsDag(libFolder,libName,'snappy',outMapFolder,fspDof=specialDof,aoiExtent=None)
        if dag is None:
            tileTifs = None
        else:
            # visualize DAG
            # visualize(dag)
            # Compute DAG
            tileTifs = client.get(dag, dagRoot)
            if not tileTifs: # list is empty
                tileTifs =  None
    else:
        print(f'Map {libName} ...')
        tileTifs = MapFloodDepthWithTiles(libFolder,libName,'snappy',outMapFolder,fspDof=specialDof,aoiExtent=None)
    print(f'Actual mapped tiles: {tileTifs}')

    # Mosaic all the tiles from a library into one tif file
    if mosaicTiles and not(tileTifs is None):
        print('Mosaic tile maps ...')
        mosaicTifName = libName+'_'+outMapFolderName+'.tif'
        # Simplest implementation, may crash with very large raster
        print(MosaicGtifs('',tileTifs,mosaicTifName,keepTifs=True))
    
    # check time
    endTime = time.time()
    usedTime = round((endTime-startTime)/60,3)
    libTime[libName] = usedTime
    # print(f'{libName} processing time (minutes):', usedTime)

# Show processing time
# Individual lib processing time
print('Individual library mapping time:', libTime)
# total time
endTimeAllLibs = time.time()
print('Total processing time (minutes):', round((endTimeAllLibs-startTimeAllLibs)/60,3))

#
# Shutdown local clusters
#
if useLocalCluster:
    print('Shutdown LocalCluster ...')
    cluster.close()
    client.shutdown()
    client.close()
    useLocalCluster = False

Tiled FLDPLN library folder: data\tmp\tile
Map folder: data\maps\mindtf_0905
Libraries to map: ['']
Map  ...
Tiles need to be mapped: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
Actual mapped tiles: ['data\\maps\\mindtf_0905\\_tile_1.tif', 'data\\maps\\mindtf_0905\\_tile_2.tif', 'data\\maps\\mindtf_0905\\_tile_3.tif', 'data\\maps\\mindtf_0905\\_tile_4.tif', 'data\\maps\\mindtf_0905\\_tile_5.tif', 'data\\maps\\mindtf_0905\\_tile_6.tif', 'data\\maps\\mindtf_0905\\_tile_7.tif', 'data\\maps\\mindtf_0905\\_tile_8.tif', 'data\\maps\\mindtf_0905\\_tile_9.tif', 'data\\maps\\mindtf_0905\\_tile_10.tif', 'data\\maps\\mindtf_0905\\_tile_11.tif', 'data\\maps\\mindtf_0905\\_tile_12.tif', 'data\\maps\\mindtf_0905\\_tile_13.tif', 'data\\maps\\mindtf_0905\\_tile_14.tif', 'data\\maps\\mindtf_0905\\_tile_15.tif', 'data\\maps\\mindtf_0905\\_tile_16.tif', 'data\\maps\\mindtf_0905\\_tile_17.tif', 'data\\maps\\mindtf_0905\\_tile_18.tif']
Mosaic tile maps ...
_mindtf_0905.tif
Individual li